# Board Game Geek Data Scraper

The boardgame data are scraped from the BoardGameGeek XML 2.0 API. There is a limit on how many items per request can be fetched but no call limit in a given time. The scraper always requests 100 boardgames at once and stores them in a CSV.

The whole scraping takes about 1h.

In [3]:
import requests
import csv
import itertools
import time
import xml.etree.ElementTree as ET

import html
import uuid
from datetime import datetime

In [11]:
# variable from 1 to 43098 comma separated
numbers = ','.join(str(i) for i in range(1, 43099))

# Split numbers into chunks of n
def grouper(iterable, n):
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args)

id_chunks = grouper(numbers.split(','), 100)

# Open CSV file for writing
with open('bgg_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Image', 'Description', 'Min Players', 'Max Players', 'Year Published', 'Min Playtime', 'Max Playtime'])

    # Loop through each chunk of ids
    for chunk in id_chunks:
        # Remove any None values from the chunk
        chunk = [id for id in chunk if id is not None]

        # Join the ids back into a comma-separated string
        ids = ','.join(chunk)

        # Send GET request to API endpoint for the current chunk of ids
        response = requests.get(f'https://boardgamegeek.com/xmlapi2/thing?type=boardgame&id={ids}')

        # Parse response content
        root = ET.fromstring(response.content)

        # Loop through each item in the response and write the requested attributes to the CSV file
        for item in root.findall('item'):
            name = item.find('name[@type="primary"]').attrib['value'] if item.find('name[@type="primary"]') is not None else ''
            image = item.find('image').text if item.find('image') is not None else ''
            description = item.find('description').text if item.find('description') is not None else ''
            minplayers = item.find('minplayers').attrib['value'] if item.find('minplayers') is not None else ''
            maxplayers = item.find('maxplayers').attrib['value'] if item.find('maxplayers') is not None else ''
            yearpublished = item.find('yearpublished').attrib['value'] if item.find('yearpublished') is not None else ''
            minplaytime = item.find('minplaytime').attrib['value'] if item.find('minplaytime') is not None else ''
            maxplaytime = item.find('maxplaytime').attrib['value'] if item.find('maxplaytime') is not None else ''
            writer.writerow([name, image, description, minplayers, maxplayers, yearpublished, minplaytime, maxplaytime])



# SQL Insert Generator

After the CSV file is created, we can use the following Python script to generate SQL INSERT statements for each row in the CSV file. AS there is a limit on flyway migration script size, we spilt these up into 10000 insert statements per script.

In [4]:

# Read the data from the CSV file
with open('bgg_data.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row

    # Open SQL file for writing
    with open('insert_data.sql', mode='w', encoding='utf-8') as sql_file:
        # Loop through each row in the CSV file
        for row in reader:
            # Generate UUID based on the name
            name = row[0]
            id = str(uuid.uuid5(uuid.NAMESPACE_DNS, name))

            # Generate current timestamp for date_created and last_updated fields
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            # Extract the values from the row
            name, image, description, minplayers, maxplayers, *_ = row  # Use * to capture additional values

            # Translate special values in name
            name = html.unescape(name)

            # Write the insert statement to the SQL file
            sql_file.write(f"INSERT INTO games (id, name, description, min_player, max_player, image_url, date_created, last_updated) VALUES ('{id}', '{name}', $${description}$$, {minplayers}, {maxplayers}, '{image}', '{current_time}', '{current_time}');\n")
